In [6]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [21]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_cars


In [22]:
ls

data/  day1_meta.ipynb  day2_visualisation.ipynb  LICENSE  README.md


Wczytywanie danych

In [0]:
df = pd.read_hdf('data/car.h5')

In [24]:
df.shape

(106494, 155)

In [25]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

Dummy model

In [28]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [29]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [30]:
df.price_currency.value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [31]:
df.price_currency.value_counts(normalize=True)

PLN    0.998084
EUR    0.001916
Name: price_currency, dtype: float64

In [0]:
df = df[df['price_currency'] != 'EUR']

In [33]:
df.shape

(106290, 155)

In [34]:
df.sample(10)

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
83698,"[Osobowe, Opel, Astra, Opel Astra J (2009-2015)]","07:24, 4 marca 2018",PLN,"Cena Brutto, Do negocjacji",26700.0,"Koło, kolski, Wielkopolskie",None,Osoba prywatna,True,False,True,True,True,True,True,False,True,True,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,...,None,None,Osoby prywatnej,110 KM,None,None,None,None,Tak,Diesel,None,None,None,None,None,None,None,Astra,1 598 cm3,None,None,None,None,None,None,None,None,None,Szary,None,None,Używane,J (2009-2015),None,None,None,None,None,None,83698
46887,"[Osobowe, Peugeot, 407]","16:55, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",9500.0,"Lublin, Lubelskie",None,Osoba prywatna,False,False,False,True,False,True,True,False,True,True,True,True,False,False,False,False,False,True,True,True,False,False,False,True,True,False,False,False,False,True,True,False,...,None,None,Osoby prywatnej,109 KM,None,Holandia,None,None,None,Diesel,None,None,None,None,None,None,None,407,1 596 cm3,None,None,None,None,None,None,None,None,None,Szary,None,None,Używane,None,None,None,None,Tak,None,None,46887
39140,"[Osobowe, Renault, Kangoo, Renault Kangoo I (1...","19:28, 28 lutego 2018",PLN,"Cena Brutto, Do negocjacji",14800.0,"Stargard Szczeciński, stargardzki, Zachodniopo...",None,Osoba prywatna,False,False,False,False,False,False,True,False,True,False,True,True,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,None,Tak,Osoby prywatnej,85 KM,None,Polska,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,Kangoo,1 200 cm3,None,None,None,None,None,None,None,None,None,Biały,None,None,Używane,I (1998-2008),None,None,None,Tak,None,None,39140
150751,"[Osobowe, BMW, Seria 3, 318, BMW Seria 3 E90 (...","20:56, 4 marca 2018",PLN,"Cena Brutto, Do negocjacji",28900.0,"Zaręby Bolędy - 18-220 Czyżew, wysokomazowieck...",AUTO JASKO,Dealer,True,False,True,True,False,True,False,False,True,True,True,True,False,False,True,True,False,True,True,True,False,False,False,False,False,False,False,False,True,True,True,False,...,None,None,Firmy,143 KM,None,Belgia,None,None,None,Diesel,None,None,None,None,318,None,None,Seria 3,2 000 cm3,None,None,No

In [35]:
for feat in df.columns:
  print(feat)

breadcrumb
created_at
price_currency
price_details
price_value
seller_address
seller_name
seller_type
feature_czujniki-parkowania-przednie
feature_poduszka-powietrzna-chroniąca-kolana
feature_kurtyny-powietrzne
feature_klimatyzacja-dwustrefowa
feature_światła-led
feature_czujnik-zmierzchu
feature_elektrycznie-ustawiane-lusterka
feature_asr-(kontrola-trakcji)
feature_poduszka-powietrzna-kierowcy
feature_cd
feature_elektryczne-szyby-przednie
feature_poduszka-powietrzna-pasażera
feature_system-start-stop
feature_światła-do-jazdy-dziennej
feature_komputer-pokładowy
feature_elektryczne-szyby-tylne
feature_klimatyzacja-manualna
feature_tapicerka-welurowa
feature_czujnik-deszczu
feature_światła-przeciwmgielne
feature_ogrzewanie-postojowe
feature_radio-niefabryczne
feature_regulowane-zawieszenie
feature_ogranicznik-prędkości
feature_zmieniarka-cd
feature_szyberdach
feature_isofix
feature_asystent-pasa-ruchu
feature_alufelgi
feature_bluetooth
feature_nawigacja-gps
feature_asystent-parkowania
fe

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list):
    continue

  factorize_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorize_values
  else:
    df[feat + SUFFIX_CAT] = factorize_values

In [45]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

print(len(cat_feats))
cat_feats

151


['created_at__cat',
 'seller_address__cat',
 'seller_name__cat',
 'seller_type__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_poduszka-powietrzna-chroniąca-kolana__cat',
 'feature_kurtyny-powietrzne__cat',
 'feature_klimatyzacja-dwustrefowa__cat',
 'feature_światła-led__cat',
 'feature_czujnik-zmierzchu__cat',
 'feature_elektrycznie-ustawiane-lusterka__cat',
 'feature_asr-(kontrola-trakcji)__cat',
 'feature_poduszka-powietrzna-kierowcy__cat',
 'feature_cd__cat',
 'feature_elektryczne-szyby-przednie__cat',
 'feature_poduszka-powietrzna-pasażera__cat',
 'feature_system-start-stop__cat',
 'feature_światła-do-jazdy-dziennej__cat',
 'feature_komputer-pokładowy__cat',
 'feature_elektryczne-szyby-tylne__cat',
 'feature_klimatyzacja-manualna__cat',
 'feature_tapicerka-welurowa__cat',
 'feature_czujnik-deszczu__cat',
 'feature_światła-przeciwmgielne__cat',
 'feature_ogrzewanie-postojowe__cat',
 'feature_radio-niefabryczne__cat',
 'feature_regulowane-zawieszenie__cat',
 'feature_

In [46]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19650.45289201444

In [49]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=2020).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.1927 ± 0.0071,param_faktura-vat__cat
0.1892 ± 0.0047,param_napęd__cat
0.1805 ± 0.0050,param_stan__cat
0.1523 ± 0.0059,param_rok-produkcji__cat
0.0937 ± 0.0027,param_skrzynia-biegów__cat
0.0615 ± 0.0027,param_moc__cat
0.0430 ± 0.0014,feature_kamera-cofania__cat
0.0197 ± 0.0044,param_pojemność-skokowa__cat
0.0163 ± 0.0004,feature_bluetooth__cat
0.0108 ± 0.0003,feature_światła-led__cat
